# WEEK 4
## TAKE HOME PROBLEMS




In [1]:
%pip install faker

Note: you may need to restart the kernel to use updated packages.


In [2]:
import sqlite3
from faker import Faker
from random import randint, uniform
from datetime import datetime, timedelta

### TABLE INFO

| Table Name | Columns |
|------------|---------|
| **SALES**  | Date, Order_id, Item_id, Customer_id, Quantity, Revenue |
| **ITEMS**  | Item_id, Item_name, Price, Department |
| **CUSTOMERS** | Customer_id, First_name, Last_name, Address |


In [3]:
fake = Faker()

conn = sqlite3.connect('sales_db.sqlite')
cursor = conn.cursor()

cursor.execute('DROP TABLE IF EXISTS CUSTOMERS')
cursor.execute('DROP TABLE IF EXISTS ITEMS')
cursor.execute('DROP TABLE IF EXISTS SALES')

cursor.execute('''
    CREATE TABLE CUSTOMERS (
        customer_id INTEGER PRIMARY KEY,
        first_name TEXT,
        last_name TEXT,
        address TEXT
    )
''')

cursor.execute('''
    CREATE TABLE ITEMS (
        Item_id INTEGER PRIMARY KEY,
        Item_name TEXT,
        price REAL,
        department TEXT
    )
''')

cursor.execute('''
    CREATE TABLE SALES (
        Date TEXT,
        Order_id INTEGER,
        Item_id INTEGER,
        Customer_id INTEGER,
        Quantity INTEGER,
        Revenue REAL,
        FOREIGN KEY (Item_id) REFERENCES ITEMS(Item_id),
        FOREIGN KEY (Customer_id) REFERENCES CUSTOMERS(customer_id)
    )
''')

def random_date(start, end):
    return start + timedelta(days=randint(0, (end - start).days))

def insert_customers(n):
    customers = []
    for i in range(1, n+1):
        customers.append((i, fake.first_name(), fake.last_name(), fake.address()))
    cursor.executemany('INSERT INTO CUSTOMERS (customer_id, first_name, last_name, address) VALUES (?, ?, ?, ?)', customers)

def insert_items():
    items = [
        (1, 'Laptop', 1000, 'Electronics'),
        (2, 'Phone', 800, 'Electronics'),
        (3, 'Chair', 150, 'Furniture'),
        (4, 'Table', 300, 'Furniture'),
        (5, 'Book', 20, 'Books'),
        (6, 'Headphones', 200, 'Electronics')
    ]
    cursor.executemany('INSERT INTO ITEMS (Item_id, Item_name, price, department) VALUES (?, ?, ?, ?)', items)

def insert_sales(n):
    start_date = datetime(2022, 1, 1)
    end_date = datetime(2023, 12, 31)
    sales = []
    
    for i in range(1, n+1):
        date = random_date(start_date, end_date).strftime('%Y-%m-%d')
        order_id = i
        item_id = randint(1, 6)
        customer_id = randint(1, 100)
        quantity = randint(1, 5)
        revenue = round(quantity * uniform(20, 1000), 2)
        sales.append((date, order_id, item_id, customer_id, quantity, revenue))
    
    cursor.executemany('INSERT INTO SALES (Date, Order_id, Item_id, Customer_id, Quantity, Revenue) VALUES (?, ?, ?, ?, ?, ?)', sales)

insert_customers(100)  
insert_items()         
insert_sales(500)      

conn.commit()


1. Pull total number of orders that were completed on 18th March 2023
2. Pull total number of orders that were completed on 18th March 2023 with the first name ‘John’ and last name Doe’
3. Pull total number of customers that purchased in January 2023 and the average amount spend per customer
4. Pull the departments that generated less than $600 in 2022
5. What is the most and least revenue we have generated by an order
6. What were the orders that were purchased in our most lucrative order

In [4]:

# Queries
queries = {
    'Total orders on 18th March 2023': '''
        SELECT COUNT(Order_id) AS total_orders
        FROM SALES
        WHERE Date = '2023-03-18';
    ''',
    
    'Total orders on 18th March 2023 by John Doe': '''
        SELECT COUNT(SALES.Order_id) AS total_orders
        FROM SALES
        JOIN CUSTOMERS ON SALES.Customer_id = CUSTOMERS.customer_id
        WHERE SALES.Date = '2023-03-18'
        AND CUSTOMERS.first_name = 'John'
        AND CUSTOMERS.last_name = 'Doe';
    ''',
    
    'Customers and avg spending in Jan 2023': '''
        SELECT COUNT(DISTINCT SALES.Customer_id) AS total_customers, 
               AVG(SALES.Revenue) AS avg_spent_per_customer
        FROM SALES
        WHERE Date BETWEEN '2023-01-01' AND '2023-01-31';
    ''',
    
    'Departments generating less than $600 in 2022': '''
        SELECT ITEMS.department, SUM(SALES.Revenue) AS total_revenue
        FROM SALES
        JOIN ITEMS ON SALES.Item_id = ITEMS.Item_id
        WHERE Date BETWEEN '2022-01-01' AND '2022-12-31'
        GROUP BY ITEMS.department
        HAVING SUM(SALES.Revenue) < 600;
    ''',
    
    'Most and least revenue by an order': '''
        SELECT MAX(Revenue) AS max_revenue, MIN(Revenue) AS min_revenue
        FROM SALES;
    ''',
    
    'Orders in most lucrative order': '''
        WITH most_lucrative_order AS (
            SELECT Order_id, SUM(Revenue) AS total_revenue
            FROM SALES
            GROUP BY Order_id
            ORDER BY total_revenue DESC
            LIMIT 1
        )
        SELECT * 
        FROM SALES
        WHERE Order_id = (SELECT Order_id FROM most_lucrative_order);
    '''
}

for query_name, query in queries.items():
    print(f"\n-- {query_name} --")
    cursor.execute(query)
    result = cursor.fetchall()
    for row in result:
        print(row)

conn.close()


-- Total orders on 18th March 2023 --
(0,)

-- Total orders on 18th March 2023 by John Doe --
(0,)

-- Customers and avg spending in Jan 2023 --
(21, 1678.7078260869564)

-- Departments generating less than $600 in 2022 --

-- Most and least revenue by an order --
(4985.57, 30.98)

-- Orders in most lucrative order --
('2022-01-31', 225, 6, 30, 5, 4985.57)
